In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "5호모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'5호모델_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="최대주주변경카운트", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '불성실공시법인지정카운트', '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+최대주주변경카운트"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트"
                                 ,plus_two="상호변경카운트+업종변경카운트", return_origianl=False)
concat_together9["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액"
                                 ,plus_two="타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액", return_origianl=False)
concat_together10["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       'BW금액+CB금액+유상증자금액', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [19]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액"
                                 ,plus_two="BW금액+CB금액+유상증자금액", return_origianl=False)
print(concat_together11["2012-01"].columns.shape)
concat_together11["2012-01"].columns

(14,)


Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [20]:
drop_togehter = tools.dict_data_drop(concat_together11.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [21]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 13.91it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 53,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=53)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 20,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=20)},
 '2014-01': {'num_of_cluseer': 3,
  'random_state': 94,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=94)},
 '2015-01': {'num_of_cluseer': 3,
  'random_state': 13,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=13)},
 '2016-01': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2017-01': {'num_of_cluseer': 3,
  'random_state': 33,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=33)},
 '2018-01': {'num_of_cluseer': 3,
  'random_state': 74,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=74)},
 '2019-01': {'num_of_cluseer': 3,
  'random_state': 67,
  'model': KMeans(max_iter=50

## 모델기록


In [22]:
size =evaluate.eval_cluster_size(model_save_dict)
#size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

747.0
485.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,610.0,497.0,587.0,533.0,519.0,586.0,607.0,571.0,550.0,715.0,582.0
1,499.0,502.0,497.0,507.0,573.0,536.0,573.0,624.0,686.0,605.0,747.0
2,485.0,606.0,536.0,619.0,639.0,665.0,685.0,724.0,741.0,706.0,727.0
기업수,1594.0,1605.0,1620.0,1659.0,1731.0,1787.0,1865.0,1919.0,1977.0,2026.0,2056.0


In [23]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
#eval_one.to_csv("{}_관리종목.csv".format(model_name))
#eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))
eval_one

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.132075,0.914286,0.023256,0.880952,0.023256,0.826923,0.837209,0.94382,0.011494,0.861538
1,0.811321,0.085714,0.906977,0.023810,0.906977,0.038462,0.046512,0.00000,0.885057,0.030769
2,0.056604,0.000000,0.069767,0.095238,0.069767,0.134615,0.116279,0.05618,0.103448,0.107692
포착군집,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000


In [24]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
#eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
#eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [25]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
#eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
#eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [26]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8932145315786151
1.0


0.10541139240506331

In [27]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

#dist.to_csv("{}_거리분해.csv".format(model_name))
#dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

매출총이익                                                                                      0.161522
총자본                                                                                        0.139662
영업이익                                                                                       0.129126
매출채권                                                                                       0.105981
당기순이익                                                                                      0.104593
판매 및 관리비                                                                                   0.100719
영업현금흐름                                                                                     0.081671
최대주주비율                                                                                     0.072469
경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트                                   0.025672
차입금의존도                                                                                     0.020760


예측

In [28]:
data2022= scaled_together["2022-01"]
mask = model_save_dict["2021-01"]["model"].predict(data2022) == 0

idx = data2022.loc[mask].index
idx

s = list_data["2022-01"].loc[idx].mean()
s.to_csv("끝")

c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


Index(['A000040', 'A000650', 'A000890', 'A000950', 'A001000', 'A001070',
       'A001140', 'A001210', 'A001360', 'A001420',
       ...
       'A357580', 'A357880', 'A361570', 'A365590', 'A365900', 'A368770',
       'A369370', 'A373200', 'A376180', 'A377030'],
      dtype='object', length=706)

In [29]:
s = list_data["2022-01"].loc[idx].mean()
s.to_csv("끝")

In [30]:
#name = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\기업명_기업코드.csv", index_col=0)

In [31]:
#pd.merge(name, pick50, left_on="기업공시코드",right_on="index")["회사명"].to_clipboard()

In [32]:
dict = evaluate.see_cluster_info(model_save_dict, eval_one, drop_togehter)

data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)
data.columns = col
#data.T.to_csv("{}_평균정보.csv".format(model_name))
#data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))

In [33]:
data.T.loc["2021-01"].T.to_csv("2021-01")

In [34]:
# 그림뽑기

#test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [35]:
#MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

In [36]:
# 포착 확률 구하기
key = list(model_save_dict.keys())
key_t1 = key[:-2]
key_t2 = key[1:-1]

return_predict_dict = {}

for t1,t2 in zip(key_t1,key_t2):
    model_t1 = model_save_dict[t1]["model"]
    data = scaled_together[t2]
    #predict_label = model_t1.predict(scaled_together[t2])
    # t2기에 대한 예측값을 담고 있다
    prob, org_prob = Optimizer.eval(model_t1,data,label_data_pp_war[t2])
    return_predict_dict[t2] = prob

return_df = pd.DataFrame()

for t,data in return_predict_dict.items():
    df = pd.DataFrame(data.values(), data.keys(), columns=[str(t)+"기_예측"])
    return_df = pd.concat([return_df,df], axis=1)
return_df

print(return_df.max(axis=0).mean())
print(return_df.max(axis=0))


return_dict = {}

label_t = eval_two.iloc[-1, :-1]
label_t2 = eval_two.iloc[-1,1:]

for labelt, labelt2, t1,t2 in zip(label_t, label_t2, key_t1,key_t2): 
    mask = model_save_dict[t1]["model"].predict(scaled_together[t2]) == labelt # t기 라벨 사용
    list_corp = scaled_together[t2].loc[mask].index.to_list()

    mask2 = model_save_dict[t2]["model"].labels_ == labelt2 # t+1기 라벨 사용
    list_corp2 = scaled_together[t2].loc[mask2].index.to_list()

    se = set(list_corp+list_corp2)
    return_dict[t2] = len(list_corp)/ len(se)

0.8997420330201279
2013-01기_예측    0.866667
2014-01기_예측    0.954545
2015-01기_예측    0.937500
2016-01기_예측    0.928571
2017-01기_예측    0.967742
2018-01기_예측    0.840909
2019-01기_예측    0.912281
2020-01기_예측    0.880952
2021-01기_예측    0.808511
dtype: float64


c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\ba

In [37]:
np.mean(list(return_dict.values()))

0.9487047094873097